In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [22]:
# Read CSV file
data = pd.read_csv('melb_data.csv')

# Drop useless features
data.drop(['Address', 'Lattitude', 'Longtitude'], axis=1, inplace=True)

# Convert Date to Year and Month
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data.drop('Date', axis=1, inplace=True)

categorical_cols = ['Suburb', 'Type', 'Method', 'SellerG', 'Regionname', 'CouncilArea', 'Postcode', 'Propertycount', 'Rooms', 'Year', 'Month', 'YearBuilt']
# Replace categorical NA with -
for col in data.columns:
    if col in categorical_cols:
        if col == 'YearBuilt':
            data[col].fillna(0, inplace=True)
        else:
            data[col].fillna('-', inplace=True)
# Replace numerical NA with mean
for col in data.columns:
    if col not in categorical_cols:
        data[col].fillna(data[col].mean(), inplace=True)

data.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Regionname,Propertycount,Year,Month
0,Abbotsford,2,h,1480000.0,S,Biggin,2.5,3067.0,2.0,1.0,1.0,202.0,151.96765,0.0,Yarra,Northern Metropolitan,4019.0,2016,12
1,Abbotsford,2,h,1035000.0,S,Biggin,2.5,3067.0,2.0,1.0,0.0,156.0,79.00000,1900.0,Yarra,Northern Metropolitan,4019.0,2016,2
2,Abbotsford,3,h,1465000.0,SP,Biggin,2.5,3067.0,3.0,2.0,0.0,134.0,150.00000,1900.0,Yarra,Northern Metropolitan,4019.0,2017,3
3,Abbotsford,3,h,850000.0,PI,Biggin,2.5,3067.0,3.0,2.0,1.0,94.0,151.96765,0.0,Yarra,Northern Metropolitan,4019.0,2017,3
4,Abbotsford,4,h,1600000.0,VB,Nelson,2.5,3067.0,3.0,1.0,2.0,120.0,142.00000,2014.0,Yarra,Northern Metropolitan,4019.0,2016,6


In [23]:
mappings = {}
# Convert categorical features to numeric
for col in categorical_cols:
    le = LabelEncoder()
    le.fit(data[col])
    mappings[col] = dict(zip(le.classes_, le.transform(le.classes_)))
    data[col] = le.transform(data[col])
data

# Pickle mappings
import pickle
with open('mappings.pkl', 'wb') as f:
    pickle.dump(mappings, f)


In [11]:
X_scaler = StandardScaler()
X_scaled = X_scaler.fit_transform(data.drop('Price', axis=1))
X_train, X_test, y_train, y_test = train_test_split(X_scaled, data['Price'], test_size=0.2, random_state=42)
print ("X_train.shape = ", X_train.shape)
print ("y_train.shape = ", y_train.shape)
print ("X_val.shape = ", X_test.shape)
print ("y_val.shape = ", y_test.shape)

X_train.shape =  (10864, 18)
y_train.shape =  (10864,)
X_val.shape =  (2716, 18)
y_val.shape =  (2716,)


In [12]:
def custom_loss_function(y_true, y_pred):    
    y_diff = tf.abs(tf.math.log(1+y_true) - tf.math.log(1+y_pred))    
    return tf.reduce_sum(tf.square(y_diff)) / tf.cast(tf.size(y_diff), tf.float32)

In [13]:
model2 = tf.keras.Sequential([
    layers.Dense(1024, activation='relu'),#, input_shape=[len(X_train.keys())]),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])
model2.compile(optimizer='adam', loss=custom_loss_function, metrics=['mae', 'mse'])

In [14]:
model2.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1)

Epoch 1/100
679/679 [==============================] - 2s 3ms/step - loss: 8.4296 - mae: 611389.6875 - mse: 730035388416.0000
Epoch 2/100
679/679 [==============================] - 2s 3ms/step - loss: 0.1377 - mae: 312091.0000 - mse: 259152510976.0000
Epoch 3/100
679/679 [==============================] - 2s 2ms/step - loss: 0.1007 - mae: 269554.8125 - mse: 210386534400.0000
Epoch 4/100
679/679 [==============================] - 2s 2ms/step - loss: 0.0892 - mae: 255061.5781 - mse: 190349197312.0000
Epoch 5/100
679/679 [==============================] - 2s 2ms/step - loss: 0.0835 - mae: 248394.4844 - mse: 212301922304.0000
Epoch 6/100
679/679 [==============================] - 2s 2ms/step - loss: 0.0803 - mae: 244562.1250 - mse: 189572349952.0000
Epoch 7/100
679/679 [==============================] - 2s 2ms/step - loss: 0.0779 - mae: 241886.4688 - mse: 242314788864.0000
Epoch 8/100
679/679 [==============================] - 2s 2ms/step - loss: 0.0768 - mae: 240339.6250 - mse: 1965120716

In [15]:
y_pred = model2.predict(X_test)
print(y_test.to_numpy(), y_pred)

85/85 [==============================] - 0s 951us/step
[2600000.  620000. 1000000. ... 2450000. 1155000. 1040000.] [[2164712.8 ]
 [ 636383.06]
 [ 723843.  ]
 ...
 [2567189.2 ]
 [1505668.4 ]
 [ 764946.5 ]]


In [16]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(100,activation='relu'))
model.add(tf.keras.layers.Dense(50,activation='sigmoid'))
model.add(tf.keras.layers.Dense(50,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [17]:
model.compile(optimizer='adam',
              loss=custom_loss_function)
history = model.fit(X_train,y_train,
                    batch_size=32,
                    epochs=250,
                    validation_data=(X_test,y_test))

Epoch 1/250
340/340 [==============================] - 1s 3ms/step - loss: 95.4052 - val_loss: 59.6155
Epoch 2/250
340/340 [==============================] - 1s 3ms/step - loss: 44.9292 - val_loss: 34.4973
Epoch 3/250
340/340 [==============================] - 1s 3ms/step - loss: 28.4359 - val_loss: 23.6235
Epoch 4/250
340/340 [==============================] - 1s 3ms/step - loss: 20.2209 - val_loss: 17.3720
Epoch 5/250
340/340 [==============================] - 1s 3ms/step - loss: 15.1447 - val_loss: 13.2361
Epoch 6/250
340/340 [==============================] - 1s 3ms/step - loss: 11.6544 - val_loss: 10.2843
Epoch 7/250
340/340 [==============================] - 1s 3ms/step - loss: 9.1048 - val_loss: 8.0777
Epoch 8/250
340/340 [==============================] - 1s 3ms/step - loss: 7.1696 - val_loss: 6.3784
Epoch 9/250
340/340 [==============================] - 1s 3ms/step - loss: 5.6650 - val_loss: 5.0447
Epoch 10/250
340/340 [==============================] - 1s 3ms/step - loss: 4.4

In [18]:
y_pred = model.predict(X_test).flatten()
print(y_test.to_numpy(), y_pred)

# calculate MAE of the model
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print('MAE: %.3f' % mae)

85/85 [==============================] - 0s 857us/step
[2600000.  620000. 1000000. ... 2450000. 1155000. 1040000.] [1928604.5   650264.56  961454.56 ... 2438927.8  1373157.    815144.3 ]
MAE: 189003.553


In [19]:
model.save('Model1.h5')
model2.save('Model2.h5')